In [4]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install YOLO

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl (11.0 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- ------------------- 2/4 [pandas]
   -------------------- ------------------- 2/4 [pandas]
   -------------------- ------------------- 2/4 [pandas]
   -------------------- ------------------- 2/4 [pandas]
   -------------------- ------------------- 2/4 [p

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, regularizers
from ultralytics import YOLO
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
import numpy as np
import time


In [ ]:
image_size = (300, 300)
batch_size = 32

train_ds = keras.utils.image_dataset_from_directory(
    "./dataset/train",
    image_size=image_size,
    batch_size=batch_size,
    color_mode="rgb" 
)
val_ds = keras.utils.image_dataset_from_directory(
    "./dataset/valid",
    image_size=image_size,
    batch_size=batch_size,
    color_mode="rgb" 
)
test_ds = keras.utils.image_dataset_from_directory(
    "./dataset/test",
    image_size=image_size,
    batch_size=batch_size,
    color_mode="rgb" 
)

Found 2526 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


In [11]:
Autotune = tf.data.AUTOTUNE

def prepare(ds ,shuffle=False):
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.prefetch(buffer_size=Autotune)
    return ds

train_ds = prepare(train_ds, shuffle = True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

In [ ]:
model = YOLO("yolov8m-cls.pt")

# Train with SGD optimizer
results = model.train(
    data="./dataset",          # path to dataset root
    epochs=400,                # longer training to reach 385+ epochs like paper
    imgsz=224,
    batch=64,
    optimizer="SGD",           # switch to SGD
    lr0=0.001,                 # initial learning rate
    momentum=0.97,             # momentum
    weight_decay=0.0005,       # typical for SGD
    augment=True,              # stronger augmentations
    flipud=0.5,
    fliplr=0.5,
    scale=0.5,
    degrees=15,
    patience=50,               # early stopping patience
    project="autism_yolov8_sgd",
    name="yolov8m_sgd_400e"
)

Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/ahd-dataset/dataset, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=400, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-cls.pt, momentum=0.97, mosaic=1.0, multi_scale=False, name=yolov8m_sgd_400e2, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=50, perspective=0.0, 

In [14]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /kaggle/input/ahd-dataset/dataset/train... found 2526 images in 2 classes ✅ 
val: /kaggle/input/ahd-dataset/dataset/valid... found 200 images in 2 classes ✅ 
test: /kaggle/input/ahd-dataset/dataset/test... found 200 images in 2 classes ✅ 
val: Fast image access ✅ (ping: 0.1±0.2 ms, read: 96.8±67.5 MB/s, size: 43.4 KB)
val: Scanning /kaggle/input/ahd-dataset/dataset/valid... 200 images, 0 corrupt: 100% ━━━━━━━━━━━━ 200/200 1.0Kit/s 0.2s
WARNING ⚠️ val: Cache directory /kaggle/input/ahd-dataset/dataset is not writeable, cache not saved.
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 13/13 8.7it/s 1.5s
                   all        0.9          1
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val
ultralytics.utils.metrics.C

In [ ]:
# Load best trained model
model = YOLO("./runs/classify/train/weightsYOLOv8/best.pt")

# Evaluate on test folder
metrics = model.val(data="./dataset/test")  # <-- test folder with autistic/non_autistic
print(metrics)  # prints accuracy, precision, recall, etc.


Ultralytics 8.3.206  Python-3.13.7 torch-2.8.0+cpu CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
YOLOv8m-cls summary (fused): 42 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
WARNING Dataset 'split=train' not found at C:\autism_detection\dataset\test\train
Found 200 images in subdirectories. Attempting to split...
Splitting C:\autism_detection\dataset\test (2 classes, 200 images) into 80% train, 20% val...
Split complete in C:\autism_detection\dataset\test_split 
train: C:\autism_detection\dataset\test_split\train... found 200 images in 2 classes  
val: C:\autism_detection\dataset\test_split\val... found 166 images in 2 classes  
test: None...
val: Fast image access  (ping: 0.20.2 ms, read: 52.159.4 MB/s, size: 64.8 KB)
val: Scanning C:\autism_detection\dataset\test_split\val... 166 images, 0 corrupt: 100% ━━━━━━━━━━━━ 166/166 1.9Kit/s 0.1s
val: New cache created: C:\autism_detection\dataset\test_split\val.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 